# Notebook `Data Extractor`
***

In [35]:
from yarl import URL
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from pathlib import Path
from src.modules.scraping_functions import *
import polars as pl

**Note** : dans le folder `data`, tu trouveras le fichier `wine_links.csv`, il contient l'ensemble des href trouvés après le lancement du module `scraper.py`

- Dans le fichier actuel, il y en a *540*, mais en modifiant les **range** de la fonction `create_urls_browse_list` du module `scraping_functions.py`, tu peux récupérer l'ensemble des hrefs.
- `⚠` ! Pour récupérer les hrefs il faut obligatoirement faire tourner le module `scraper.py` car les hrefs sont générés post-javascript (évidemment ça ne marche pas dans les notebooks...) mais pour les caractéristiques des vins, prix et autres, c'est avant le javascript que c'est généré donc tu peux le faire ici ! 

In [36]:
root = Path(".").resolve()
data_folder = root / "data"
file_path = data_folder / "wine_links.csv"
session = create_session()

Autrement dit la fonction ci-dessous, `create_all_wine_urls` ne fonctionnera pas ici

In [23]:
# create_all_wine_urls()

Donc on importe plutôt les données du folder data en polars puis on retransforme en liste

In [76]:
all_wine_urls = list(
    chain.from_iterable(pl.read_csv(file_path, has_header=False).to_numpy().tolist())
)

In [77]:
all_wine_urls = list(set(all_wine_urls))

In [78]:
df_polars = pl.read_csv(file_path, has_header=False)
df_polars.select("column_1").group_by("column_1").count().sort("count", descending=True).to_numpy().tolist()

[['https://vinatis.com/59413-le-bal-des-papillons-2022-domaine-horgelus;',
  197],
 ['https://vinatis.com/55632-montagne-noire-2021-chateau-auzias;', 193],
 ['https://vinatis.com/60864-chateau-puy-d-amour-cuvee-grain-de-folie-2020;',
  193],
 ['https://vinatis.com/57419-villa-des-anges-reserve-2021-jeff-carrel;', 193],
 ['https://vinatis.com/61101-les-darons-2022-by-jeff-carrel;', 193],
 ['https://vinatis.com/53297-wolf-pinot-grigio-2021-tussock-jumper;', 148],
 ['https://vinatis.com/59900-viognier-2022-les-jamelles;', 99],
 ['https://vinatis.com/65066-pinot-gris-2022-sophie-schaal;', 99],
 ['https://vinatis.com/59228-colombard-sauvignon-blanc-n3-2022-domaine-uby;',
  99],
 ['https://vinatis.com/62418-grande-reserve-chardonnay-2022-maison-castel;',
  99],
 ['https://vinatis.com/56828-igp-vaucluse-rouge-xavier-vignon;', 97],
 ['https://vinatis.com/60636-chateau-malbec-2020;', 97],
 ['https://vinatis.com/60679-chateau-moulin-bellegrave-2020;', 97],
 ['https://vinatis.com/54014-cinquenoci

*Note* : il y a **BEAUCOUP** de doublons.

In [79]:
len(all_wine_urls)

4374

Enfin, on extrait tout le contenu des pages avec `extract_all_pages`. Ensuite..c'est à toi ! 🤪

In [ ]:
all_html_pages = extract_all_pages(session, all_wine_urls)

liste des caractéristiques trouvées :

- Titre
- Photo
- Contenance
- Type (Vin blanc, Vin rouge, etc)
- Région
- Volume alcool (%)
- Prix
- Adjectifs ["Fruité", "Vif", "Vignerons passionnés"]
- Millésime
- Cépage
- Gouts
- est bio
- haute valeur environementale
- vigneron indépendant
- est une nouveauté sur le site (tire peut-être le prix vers le bas)
- notes clients
- nb avis
- et bien d'autres j'imagine.....


In [5]:
all_html_pages[0]

'\n<!DOCTYPE HTML><html lang="fr"><head><meta charset="utf-8" /><title>Vin Rosé GOURMANDISE des Vignerons de Saint Tropez au meilleur prix du net !</title><meta name="description" content="Vous cherchez les rosés de votre été ? Trouvez notre sélection de vins rosés sur vinatis.com, leader en vente de vin sur internet !" /><meta name="keywords" content="gourmandise 2022, maitres vignerons de la presqu&#039;Île de saint-tropez, vin rosé, provence-alpes-côte d&#039;azur, méditerranée igp, 12,5 % vol" /><meta name="robots" content="index,follow" /><meta name="viewport" content="width=device-width, minimum-scale=0.25, maximum-scale=1.6, initial-scale=1.0" /><meta name="apple-mobile-web-app-capable" content="yes" /><meta property="og:title" content="GOURMANDISE 2022 - MAITRES VIGNERONS DE LA PRESQU\'ÎLE DE SAINT-TROPEZ"><meta property="og:image" content="https://www.vinatis.com/77926-large_default/gourmandise-2022-maitres-vignerons-de-la-presqu-ile-de-saint-tropez.png"><meta property="og:sit

## 🐱‍💻 Quelques tests, module `soup` à faire

In [34]:
content_page = all_html_pages[11]
soup = BeautifulSoup(content_page, "html.parser")
print(soup.find("h1", id="produit-titre").text)
print(soup.find_all("a", attrs={"class": "maillage"}))
print(soup.find("span", id="our_price_display").text)
print(
    soup.find_all(
        "a",
        attrs={
            "class": "text-bold padding-5.no-padding-bottom color-green top-label-shadow-green inline-block taille-xs text-uppercase"
        },
    )
)

print(
    soup.find_all(
        "div",
        attrs={"class": "margin-top-3 display-inline"},
    )
) #  pour obtenir destockage, nouveauté et bio ;)

 GRAND FERRAGE 2022 CUVEE ORSURO - MATHILDE CHAPOUTIER SELECTION
[<a class="maillage" href="/achat-vin-rose">Vin Rosé</a>, <a class="maillage" href="/achat-vin-provence">Provence</a>, <a class="maillage" href="/maisons/chateau-d-azur">Azur</a>, <a class="maillage" href="/achat-vin-cote-de-provence">Côtes de Provence</a>, <a class="maillage" href="/achat-vin-rose">Vin Rosé</a>, <a class="maillage" href="/achat-vin-provence">Provence</a>, <a class="maillage" href="/maisons/chateau-d-azur">Azur</a>, <a class="maillage" href="/achat-vin-cote-de-provence">Côtes de Provence</a>, <a class="maillage" href="/m-chapoutier">Chapoutier</a>, <a class="maillage" href="/maisons/tariquet">Tariquet</a>, <a class="maillage" href="/achat-vin">vins</a>]
  12,90 €   9,65 €    -25%   
[]
[<div class="margin-top-3 display-inline"> <a class="text-bold padding-5 no-padding-bottom color-flash-red top-label-shadow-flash-red inline-block taille-xs text-uppercase" href="/destockage">Destockage</a></div>]


In [27]:
link = soup.find("img", attrs={"class": "img-full-width img-max-450 center-block"}).get("src")

In [40]:
import pandas as pd
from IPython.display import Image, HTML

def path_to_image_html(path):
    """
    This function essentially convert the image url to 
    '<img src="'+ path + '"/>' format. And one can put any
    formatting adjustments to control the height, aspect ratio, size etc.
    within as in the below example. 
    """

    return f"<img src= '{path}' style=max-height:248px;/>"

In [41]:
df = pd.DataFrame({'col1': [1,2], 'link': [link, link]})

In [42]:
HTML(
    df.to_html(
        escape=False, formatters=dict(link=path_to_image_html)
    )
)

,col1,link
0,1,
1,2,


In [22]:
print(
    soup.find_all(
        "div",
        attrs={"class": "margin-top-5 hide-xs"},
    )
)

[<div class="margin-top-5 hide-xs"> <strong><span class="margin-right margin-bottom bg-gray-dark taille-md padding-horizontal-30 padding-vertical-5 rounded-corner-3 label">Gourmand</span></strong> <strong><span class="margin-right margin-bottom bg-gray-dark taille-md padding-horizontal-30 padding-vertical-5 rounded-corner-3 label">Fraîcheur</span></strong> <strong><span class="margin-right margin-bottom bg-gray-dark taille-md padding-horizontal-30 padding-vertical-5 rounded-corner-3 label">Valeur sûre</span></strong></div>]


In [29]:
print(
    [element.text for element in soup.find_all("span", id="quantity_discount_pretaxe")]
)

print(
    [
        element.text
        for element in soup.find_all(
            "div",
            attrs={
                "class": "table-cell-css vertical-align-top padding-vertical-5 nowrap padding-right-10 taille-xs color-gray-darker"
            },
        )
    ]
)

print(
    [
        element.text
        for element in soup.find_all(
            "div",
            attrs={
                "class": "table-cell-css vertical-align-top padding-vertical-5 taille-xs color-gray-darker text-bold"
            },
        )
    ]
)


[' 7,20 € ']
[' \xa0\xa0Millésime', ' \xa0\xa0Cépage', ' \xa0\xa0Bouchon', ' \xa0\xa0Goûts', ' \xa0\xa0Par Goûts', " \xa0\xa0A l'oeil", ' \xa0\xa0Au nez', ' \xa0\xa0En bouche', ' \xa0\xa0Température de service', ' \xa0\xa0Service', ' \xa0\xa0Conservation', ' \xa0\xa0Accords mets-vin', ' \xa0\xa0Accords recommandés']
[' 2022', ' 77% Merlot, 16% Cabernet-Franc, 7% Cabernet-Sauvignon', ' Capsule à vis', " Rosé d'apéritif", ' Fruité', ' Robe rose litchi brillante avec de légers reflets corail.', " Arômes de fruits rouges et des notes d'agrumes.", ' Souple et élégante avec une finale fraiche.', ' 8-10°C', ' En bouteille', ' A boire dans les 2 ans', ' Apéritif, Entrée, Poisson, Fromage, Dessert fruité', ' À l’apéritif ou avec des toasts de jambon Bellota, des sushis et makis, ou une tartelette aux framboises.']
